# 가사만들기

In [1]:
#initiating const value; hyper parameter
RANDOM_STATE = 77
BATCH_SIZE = 1024
NUM_WORDS = 14000

In [2]:
# loading image from filepath
import glob
import sys, os

# making tensor
import tensorflow  as tf

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# for regular expression precess
import re

# spliting data-set into train-test
from sklearn.model_selection import train_test_split

In [3]:
txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'
txt_list = glob.glob(txt_file_path)

In [4]:
raw_corpus = []

for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print(f'문장의 개수 : {len(raw_corpus)}')
raw_corpus[-5:]

문장의 개수 : 187088


['1 2 3 4. Fiasco, fiasco [Chorus:]',
 '1 2 3 4',
 "I'm calling, I'm calling, I'm calling it fiasco,",
 "I'm calling, I'm calling, I'm calling it fiasco,",
 "I'm calling, I'm calling, I'm calling it fiasco. "]

In [5]:
long_sntc = []

def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() 
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) 
    sentence = re.sub(r'[" "]+', " ", sentence) 
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) 
    sentence = sentence.strip()
    if len(sentence) >= 40:
        long_sntc.append(sentence)        
        return None
    sentence = '<start> ' + sentence + ' <end>'
    return sentence

In [6]:
corpus = []
for sentence in raw_corpus:   
    if len(sentence) == 0: continue   
   
    corpus.append(preprocess_sentence(sentence))

corpus = [i for i in corpus if i is not None]

print(len(corpus))
corpus[-5:]

108323


['<start> so . . . you then go calling <end>',
 '<start> fiasco , call it fiasco , fiasco <end>',
 '<start> . . . fiasco , fiasco , . . . fiasco <end>',
 '<start> . fiasco , fiasco chorus <end>',
 '<start>  <end>']

# 문장 길이 제한
일부분에 불과하지만 길이가 너무 긴 문장은 반복되는, 무의미한 어구로 인한 것이라 판단하였습니다.

-> sentence: str, length를 40으로 제한했습니다.

In [7]:
print(len(long_sntc))
long_sntc[20:30]

67663


['good enough for me and my bobby mcghee la da da',
 'bobby mcghee , yeah la da da da da da da',
 'bobby mcghee , yeah la da la la da da la da da la da da',
 'oh , my bobby mcghee , yeah la la la la la la la la',
 'la la la la la la la la la la la la la la la',
 'oh , my bobby mcghee , yeah well , i call him my lover , call him my man',
 'i said , i call him my lover did the best i can , c mon',
 'la da , la da , la da , la da , la da , la da , la da , la la',
 'la da , la da , la da , la da , la da , la da , la da , la',
 'hey , hey , hey , bobby mcghee , yeah didn t i make you feel like you were the only man , yeah ,']

In [8]:
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=NUM_WORDS, 
        filters=' ',
        oov_token="<unk>"
    )    
    tokenizer.fit_on_texts(corpus)
    tensor = tokenizer.texts_to_sequences(corpus)
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen=15)     
    
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)
tensor.shape

(108323, 15)

In [9]:
print(tensor.shape)

for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 15: break

(108323, 15)
1 : <unk>
2 : <start>
3 : <end>
4 : i
5 : ,
6 : the
7 : you
8 : and
9 : a
10 : to
11 : it
12 : me
13 : my
14 : t
15 : in


In [10]:
tokenizer.word_counts['i']

26795

In [11]:
src_input = tensor[:, :-1]
tgt_input = tensor[:, 1:]

In [12]:
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2, random_state=RANDOM_STATE)

enc_train.shape, enc_val.shape, dec_train.shape, dec_val.shape

((86658, 14), (21665, 14), (86658, 14), (21665, 14))

In [13]:
BUFFER_SIZE = len(src_input)

dataset_train = tf.data.Dataset.from_tensor_slices((enc_train, dec_train))
dataset_train = dataset_train.shuffle(BUFFER_SIZE)
dataset_train = dataset_train.batch(BATCH_SIZE, drop_remainder=True)

dataset_train

<BatchDataset shapes: ((1024, 14), (1024, 14)), types: (tf.int32, tf.int32)>

In [14]:
dataset_test = tf.data.Dataset.from_tensor_slices((enc_val, dec_val))
dataset_test = dataset_test.shuffle(BUFFER_SIZE)
dataset_test = dataset_test.batch(BATCH_SIZE, drop_remainder=True)

dataset_test

<BatchDataset shapes: ((1024, 14), (1024, 14)), types: (tf.int32, tf.int32)>

In [15]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
        
embedding_size = 200
hidden_size = 1024
lyricist = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [16]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

lyricist.compile(loss=loss, optimizer=optimizer)
lyricist.fit(dataset_train, epochs=3)

Epoch 1/3
84/84 [==============================] - 68s 739ms/step - loss: 3.7860
Epoch 2/3
84/84 [==============================] - 65s 769ms/step - loss: 2.8109
Epoch 3/3
84/84 [==============================] - 67s 800ms/step - loss: 2.7060


In [17]:
lyric_pred = lyricist.predict(enc_val, verbose=1, batch_size=1024)
lyricist.evaluate(lyric_pred, dec_val)

15/22 [===================>..........] - ETA: 1s

ResourceExhaustedError:  OOM when allocating tensor with shape[14336,14001] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node text_generator/dense/Tensordot/MatMul (defined at tmp/ipykernel_743/185253743.py:14) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.
 [Op:__inference_predict_function_4923]

Function call stack:
predict_function


In [18]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [25]:
generate_text(lyricist, tokenizer, init_sentence="<start> i calling", max_len=20)

'<start> i calling to the you be <end> '

In [ ]:
# 회고

시험땜에 시간이 없었습니다. ㅠ

갑자기 커널이 터져서 epoch 3회밖에 못했습니다.

predict 오류를 해결하는 것이 어렵군요.

직접 설계하는 것이 아니라 라이브러리를 사용할 때는 편리하지만

에러발생 시에 대처가 어렵다는 것을 알았습니다.